## [Neuralism](https://twitter.com/NeuralismAI) Prompt Generator with [MadLib](https://colab.research.google.com/drive/1gGwD0zDvyx0OnJU4KLjE8HrwnTlzI2rS)

This is a more advanced version of the [previous prompt generator](https://colab.research.google.com/drive/1mrlY_mc-HdIxEHILY2BvA9u0mOjD2Ze5). It can have specific prompt templates instead of one default template like from the other one.



Features:
- custom prompt template input field
- prompt weights
- vqgan/diffusion format toggle
- text file output

To do:
- add more templates


Please feel free to submit prompt templates by sending them to Thor#0897 on discord or [thors_thunder04](https://twitter.com/thors_thunder04) on twitter. Since that is what will help make this a great prompt generator. With as many templates as possible.

All current prompt templates can be found [here](https://github.com/sanzelda/prompt_gen/blob/main/templates.py)

In [ ]:
#@markdown # Download prompt lists
!git clone https://github.com/Daiten/promptmaster.git

In [ ]:
#@markdown Madlib prompt module by [@remi_durant](https://twitter.com/remi_durant)
import re
import random
import copy
import inflect
engine = inflect.engine()

def madlib_template( templates, global_lookup = {}, 
        options = { 
            "avoid_repeats" : True, 
            "fix_indefinite_articles" : True 
        }):

    # get prompt template weights and pick one
    weights = [ p.get('weight') or 1 for p in templates]
    t = random.choices( templates, weights=weights, k=1 )[0]
    
    lookup = { **global_lookup, **(t.get('lookup') or {}) }
    lookup_cardstack = {}

    def on_madlib_sub( match ):
        opt = match.group(2).split('|')

        key = opt[0]

        # do a lookup if there's only one option in the brackets
        if len(opt) == 1 and lookup.get(key):

            # discard used words to avoid repeats, unless no words remain
            if options["avoid_repeats"]:

                if len(lookup_cardstack.get(key) or []) == 0:
                    lookup_cardstack[key] = copy.copy( lookup[key] )

                g2 = random.choice( lookup_cardstack[key] )
                lookup_cardstack[key].remove(g2)

            else:
                g2 = random.choice( lookup.get(key) )
          
        # or just pick one of the given options  
        else:
            g2 = random.choice( opt )

        g1 = match.group(1)
        if g1 is not None:

            # if the previous word is 'A' or 'An', figure out if the 'n' is needed
            if options["fix_indefinite_articles"]:
                    if g2[0].lower() in list("aeiou"):
                        g1 = g1[0] + 'n'
                    else:
                        g1 = g1[0]
            g2 = g1 + ' ' + g2
            
        return g2
    
    # find madlib spots, and replace
    return re.sub(r"(\b[Aa]n? )?{([^{}]*)}", on_madlib_sub, t['prompt'] )

thing = 12

## How to add custom prompts



There are the different variables you can put in your prompts:

*   `{adjective}`
*   `{animal}`
*   `{artist}`
*   `{color}`
*   `{thing}`
*   `{shape}`
*   `{style}`
*   `{location}`
*   `{element}` [water, fire, earth, air] (not on the github)

The word lists to these variables can be found [here](https://github.com/sanzelda/prompt_gen).


There is also a way to choose a random word/phrase without these variables.
If you format a prompt like this: "It's cold `{today|tonight|in here}`, so I need a jacket." or "I need to `{sit down|eat something}`"

It will choose at random, either "today", "tonight" or "in here" for the first example. And either "sit down" or "eat something" for the second example.

---
You can also add weights to your prompts to change their frequency by adding a number after `-!-` in your template.

Here is an example of how they work:

The default weight of a prompt is 1.

If you have 2 prompt templates, one with the default weight of 1, and one with a weight of 2. The frequency of the first prompt will be `1/3` and the frequency of the second prompt will be `2/3`. 

In other words, the probability of landing on a prompt template is: `weight of the template/sum of all weights`


Example of a custom template with a weight of 3 for `custom_promt_templates`:

"There are thousands of `{animal}` in the sky -!- 3"


## Generate random prompt

In [ ]:
import random
amount_of_prompts =  100#@param {type:"integer"}
VQGAN_prompt = False #@param {type:"boolean"}

#@markdown The outputs will be saved to /content/prompt_gen_output.txt .
#@markdown Also, if you run this cell more than once, it will 
#@markdown simply add the new prompts in addition to the previous ones.
txt_file_output = False #@param {type:"boolean"}

# imports the prompt templates
from promptmaster.templates import *


prompt_lookup = {}
prompt_lookup["adjective"] = open("/content/promptmaster/adjectives.txt").read().splitlines()
prompt_lookup["animal"] = open("/content/promptmaster/animals.txt").read().splitlines()
prompt_lookup["artist"] = open("/content/promptmaster/artists.txt").read().splitlines()
prompt_lookup["color"] = open("/content/promptmaster/colors.txt").read().splitlines()
prompt_lookup["thing"] = open("/content/promptmaster/things.txt").read().splitlines()
prompt_lookup["shape"] = open("/content/promptmaster/shapes.txt").read().splitlines()
prompt_lookup["style"] = open("/content/promptmaster/styles.txt").read().splitlines()
prompt_lookup["location"] = open("/content/promptmaster/locations.txt").read().splitlines()
prompt_lookup["being"] = open("/content/promptmaster/beings.txt").read().splitlines()
prompt_lookup["of"] = open("/content/promptmaster/of_something.txt").read().splitlines()
prompt_lookup["verb"] = open("/content/promptmaster/verbs.txt").read().splitlines()
prompt_lookup["intrans"] = open("/content/promptmaster/intrans.txt").read().splitlines()
prompt_lookup["clothes"] = open("/content/promptmaster/clothes.txt").read().splitlines()
prompt_lookup["transport"] = open("/content/promptmaster/transports.txt").read().splitlines()


#@markdown ---

#@markdown Custom Prompts
#@markdown separate templates with `||` and weights to a template with `-!-`
custom_prompt_templates = "" #@param {type:"string"}


# so that it doesn't generate blanks
if custom_prompt_templates.strip() != "":
    
    custom_prompt_templates = custom_prompt_templates.split("||")
    for prompt_template in custom_prompt_templates:
        prompt_template = prompt_template.split("-!-")
        if len(prompt_template) == 2:
            prompt_templates.append({
                "prompt": prompt_template[0].strip(),
                "weight": float(prompt_template[1].strip())
                })
        else:
            prompt_templates.append({"prompt": prompt_template[0].strip()})


# these are here just for testing things, they don't do anything otherwise.
test_prompts = [
                {
                    "prompt": "tests 1 {lookup}"
                },
                {
                    "prompt": "test 2 {lookup}"
                }
]
test_lookup = {
    "lookup": ["one", "two", "three"],
    "lookdown": ["four", "five", "six"]
}

if txt_file_output:
    txtFile = open("prompt_gen_output.txt", "a")

for i in range(amount_of_prompts):

    prompt = madlib_template(prompt_templates, prompt_lookup)

    if VQGAN_prompt:
        prompt = prompt.replace(",", " |")
    
    print(prompt)
    if txt_file_output:
        txtFile.write(prompt+"\n")
        
if txt_file_output:
    txtFile.close()
